<a href="https://colab.research.google.com/github/varun999ds/GoogleMeridian/blob/main/notebooks/Meridian_Geo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/meridian/blob/main/demo/Meridian_Getting_Started.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/meridian/blob/main/demo/Meridian_Getting_Started.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# **Introduction to Meridian Demo**

Welcome to the Meridian end-to-end demo. This simplified demo showcases the fundamental functionalities and basic usage of the library, including working examples of the major modeling steps:


<ol start="0">
  <li><a href="#install">Install</a></li>
  <li><a href="#load-data">Load the data</a></li>
  <li><a href="#configure-model">Configure the model</a></li>
  <li><a href="#model-diagnostics">Run model diagnostics</a></li>
  <li><a href="#generate-summary">Generate model results & two-page output</a></li>
  <li><a href="#generate-optimize">Run budget optimization & two-page output</a></li>
  <li><a href="#save-model">Save the model object</a></li>
</ol>


Note that this notebook skips all of the exploratory data analysis and preprocessing steps. It assumes that you have completed these tasks before reaching this point in the demo.

This notebook utilizes sample data. As a result, the numbers and results obtained might not accurately reflect what you encounter when working with a real dataset.

<a name="install"></a>
## Step 0: Install

1\. Make sure you are using one of the available GPU Colab runtimes which is **required** to run Meridian. You can change your notebook's runtime in `Runtime > Change runtime type` in the menu. All users can use the T4 GPU runtime which is sufficient to run the demo colab, free of charge. Users who have purchased one of Colab's paid plans have access to premium GPUs (such as V100, A100 or L4 Nvidia GPU).

2\. Install the latest version of Meridian, and verify that GPU is available.

In [ ]:
# Install meridian: from PyPI @ latest release
!pip install --upgrade google-meridian[colab,and-cuda]

# Install meridian: from PyPI @ specific version
# !pip install google-meridian[colab,and-cuda]==1.1.1

# Install meridian: from GitHub @HEAD
# !pip install --upgrade "google-meridian[colab,and-cuda] @ git+https://github.com/google/meridian.git@main"

In [ ]:

import arviz as az
import IPython
from meridian import constants
from meridian.analysis import analyzer
from meridian.analysis import formatter
from meridian.analysis import optimizer
from meridian.analysis import summarizer
from meridian.analysis import visualizer
from meridian.data import data_frame_input_data_builder
from meridian.data import test_utils
from meridian.model import model
from meridian.model import prior_distribution
from meridian.model import spec
import numpy as np
import pandas as pd
# check if GPU is available
from psutil import virtual_memory
import tensorflow as tf
import tensorflow_probability as tfp

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print(
    'Num GPUs Available: ',
    len(tf.config.experimental.list_physical_devices('GPU')),
)
print(
    'Num CPUs Available: ',
    len(tf.config.experimental.list_physical_devices('CPU')),
)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

This notebook will run with the Regional data.

This notebok is saved in local github

<a name="load-data"></a>
## Step 1: Load the data

Load the [simulated dataset in CSV format](https://github.com/google/meridian/blob/main/meridian/data/simulated_data/csv/geo_all_channels.csv) as follows.

1\. Read the data into a Pandas DataFrame.

In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/google/meridian/refs/heads/main/meridian/data/simulated_data/csv/geo_all_channels.csv"
)

nationalPDF = pd.read_csv("https://raw.githubusercontent.com/varun999ds/GoogleMeridian/main/Simulated_Data_National.csv")

regionalPDF = pd.read_csv("https://raw.githubusercontent.com/varun999ds/GoogleMeridian/main/Simulated_Data_RF.csv")


In [ ]:
df = df.drop(columns=['Unnamed: 0'], axis = 1)
df.columns

In [ ]:
df.describe()

In [ ]:
regionalPDF = regionalPDF.drop(columns=['Unnamed: 0'], axis = 1)
regionalPDF.columns


In [ ]:
print(regionalPDF.dtypes, '\n\n')
print(regionalPDF.isnull().sum())

In [ ]:
regionalPDF.describe()

In [ ]:
print(len(regionalPDF['time'].unique()))
print(regionalPDF['time'].min(), nationalPDF['time'].max())

In [ ]:
regionalPDF = regionalPDF.sort_values(by=['time'])

#different type of columns:
spend_cols = [col for col in regionalPDF.columns if 'spend' in col]
print('spend_cols - ', spend_cols)
impression_cols = [col for col in regionalPDF.columns if 'impression' in col]
print('impression_cols - ', impression_cols)
reach_cols = [col for col in regionalPDF.columns if 'reach' in col]
print('reach_cols - ', reach_cols)
frequency_cols = [col for col in regionalPDF.columns if 'frequency' in col]
print('frequency_cols - ', frequency_cols)

In [ ]:
#Check for correlation
correlation_matrix = regionalPDF[spend_cols + impression_cols + reach_cols + frequency_cols + ['conversions']].corr()

correlation_matrix = pd.DataFrame(np.triu(correlation_matrix), index=correlation_matrix.columns, columns=correlation_matrix.columns)

plt.figure(figsize=(10, 6)) # Adjust figure size as needed
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Heatmap of Features')
plt.show()

In [ ]:
# Check for  / visualize the parameters in the mertices.
pdf = regionalPDF[spend_cols + impression_cols + reach_cols + frequency_cols + ['conversions']]
plt.figure(figsize=(9, 5))
pdf[spend_cols].boxplot()
plt.title('Box Plots of spend')
plt.ylabel('spend')
plt.show()

plt.figure(figsize=(10, 5))
pdf[impression_cols].boxplot()

plt.title('Box Plots of impressions')
plt.ylabel('impressions')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#plot conversions
plt.figure(figsize=(20, 5))
# Plot the first series
plt.plot(regionalPDF.sort_values('time')['time'].unique(), regionalPDF.groupby(['time']).sum()['conversions'], label='Series 1 (Linear)')

# # Add labels and title for clarity
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
# plt.title('Two Series on the Same Plot')

# # Add a legend to distinguish the series
# plt.legend()

# Display the plot
plt.show()

2\. Create a DataFrameInputDataBuilder instance.

In [ ]:
# builder = data_frame_input_data_builder.DataFrameInputDataBuilder(
#     kpi_type='non_revenue',
#     default_kpi_column='conversions',
#     default_revenue_per_kpi_column='revenue_per_conversion',
# )

3\. Offer the components to the builder. Note that the components may be offered all at once or piecewise.

In [ ]:
from meridian.data import data_frame_input_data_builder as data_builder

builder = data_builder.DataFrameInputDataBuilder(
    kpi_type='non_revenue',
    default_kpi_column="conversions",
    default_revenue_per_kpi_column="revenue_per_conversion",
)
builder = (
    builder
        .with_kpi(regionalPDF)
        .with_revenue_per_kpi(regionalPDF)
        .with_population(regionalPDF)
        .with_controls(regionalPDF, control_cols=["GQV", "Competitor_Discount"])
        .with_reach(
            regionalPDF,
            reach_cols = reach_cols,
            frequency_cols = frequency_cols,
            rf_spend_cols = ['Channel4_spend', 'Channel5_spend'],
            rf_channels = ['Channel4', 'Channel5'],
        )
)

channels = ["Channel0", "Channel1", "Channel2", "Channel3"]
builder = builder.with_media(
    regionalPDF,
    media_cols=[f"{channel}_impression" for channel in channels],
    media_spend_cols=[f"{channel}_spend" for channel in channels],
    media_channels=channels,
)

data = builder.build()

Note that the simulated data here does not contain reach and frequency. We recommend including reach and frequency data whenever they are available. For information about the advantages of utilizing reach and frequency, see [Bayesian Hierarchical Media Mix Model Incorporating Reach and Frequency Data](https://research.google/pubs/bayesian-hierarchical-media-mix-model-incorporating-reach-and-frequency-data/#:~:text=By%20incorporating%20R%26F%20into%20MMM,based%20on%20optimal%20frequency%20recommendations.). For code snippet for loading reach and frequency data, see [Load geo-level data with reach and frequency](https://developers.google.com/meridian/docs/user-guide/load-geo-data-with-rf)

The documentation provides guidance for instances where reach and frequency data is accessible for specific channels. Additionally, for information about how to load other data types and formats, including data with reach and frequency, see [Supported data types and formats](https://developers.google.com/meridian/docs/user-guide/supported-data-types-formats).

<a name="configure-model"></a>
## Step 2: Configure the model

Meridian uses Bayesian framework and Markov Chain Monte Carlo (MCMC) algorithms to sample from the posterior distribution.

1\. Inititalize the `Meridian` class by passing the loaded data and the customized model specification. One advantage of Meridian lies in its capacity to calibrate the model directly through ROI priors, as described in [Media Mix Model Calibration With Bayesian Priors](https://research.google/pubs/media-mix-model-calibration-with-bayesian-priors/). In this particular example, the ROI priors for all media channels are identical, with each being represented as Lognormal(0.2, 0.9).

In [ ]:
import tensorflow_probability as tfp
from meridian.data import input_data
from meridian.model import prior_distribution
from meridian.model import spec
from meridian import constants

# roi_mu = 0.2  # Mu for ROI prior for each media channel.
# roi_sigma = .9  # Sigma for ROI prior for each media channel.
# prior = prior_distribution.PriorDistribution(
#     roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
# )

#Specify the priors for media channels:
#my_input_data = input_data.InputData(kpi = regionalPDF['conversions'], kpi_type='non_revenue', population=regionalPDF['population'])
build_media_channel_args = data.get_paid_media_channels_argument_builder()

# ROI prior for channels without R&F data
roi_m = build_media_channel_args(
  Channel0=(0.5, 0.9),
  Channel1=(0.5, 0.9),
  Channel2=(0.5, 0.9),
  Channel3=(0.5, 0.9),
)
roi_m_mu, roi_m_sigma = zip(*roi_m)

# ROI prior for channels with R&F data
build_rf_channel_args = data.get_paid_rf_channels_argument_builder()
roi_rf = build_rf_channel_args(
  Channel4=(0.5, 0.8),
  Channel5=(0.5, 0.8),
)
roi_rf_mu, roi_rf_sigma = zip(*roi_rf)

prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(
        roi_m_mu, roi_m_sigma, name=constants.ROI_M
    ),
    roi_rf=tfp.distributions.LogNormal(
        roi_rf_mu, roi_rf_sigma, name=constants.ROI_RF
    ),
)
model_spec = spec.ModelSpec(prior=prior)

In [ ]:
# prior.sigma.sample()

In [ ]:
model_spec = spec.ModelSpec(
    prior=prior_distribution.PriorDistribution(),
    media_effects_dist='log_normal',
    hill_before_adstock=False,
    max_lag=8,
    unique_sigma_for_each_geo=False,
    media_prior_type='roi',
    roi_calibration_period=None,
    rf_prior_type='roi',
    rf_roi_calibration_period=None,
    organic_media_prior_type='contribution',
    organic_rf_prior_type='contribution',
    non_media_treatments_prior_type='contribution',
    knots=3,
    baseline_geo=None,
    holdout_id=None,
    control_population_scaling_id=None,
)

In [ ]:
mmm = model.Meridian(input_data=data, model_spec=model_spec)

2\. Use the `sample_prior()` and `sample_posterior()` methods to obtain samples from the prior and posterior distributions of model parameters. If you are using the T4 GPU runtime this step may take about 10 minutes for the provided data set.

In [ ]:
%%time
mmm.sample_prior(500)
mmm.sample_posterior(
    n_chains=10, n_adapt=2000, n_burnin=500, n_keep=1000, seed=0
)

For more information about configuring the parameters and using a customized model specification, such as setting different ROI priors for each media channel, see [Configure the model](https://developers.google.com/meridian/docs/user-guide/configure-model).

<a name="model-diagnostics"></a>
## Step 3: Run model diagnostics

After the model is built, you must assess convergence, debug the model if needed, and then assess the model fit.

1\. Assess convergence. Run the following code to generate r-hat statistics. R-hat close to 1.0 indicate convergence. R-hat < 1.2 indicates approximate convergence and is a reasonable threshold for many problems.

In [ ]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

In [ ]:
model_diagnostics

In [ ]:
# model_diagnostics = visualizer.ModelDiagnostics(meridian)
model_diagnostics.plot_prior_and_posterior_distribution()

2\. Assess the model's fit by comparing the expected sales against the actual sales.

In [ ]:
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

In [ ]:
model_diagnostics.predictive_accuracy_table()

In [ ]:
media_summary = visualizer.MediaSummary(mmm)
media_summary.summary_table()

In [ ]:
len(regionalPDF)

In [ ]:
baseline = model_fit.model_fit_data.baseline
# Flatten the 3D array to 1D and then reshape to 2D for DataFrame
new_var_flattened = new_var.values.reshape(-1, 1)
pd.DataFrame(new_var_flattened, columns=['baseline_value'])

In [ ]:
model_fit.model_fit_data.data_vars.variables['baseline']

For more information and additional model diagnostics checks, see [Modeling diagnostics](https://developers.google.com/meridian/docs/user-guide/model-diagnostics).

<a name="generate-summary"></a>
## Step 4: Generate model results & two-page output

To export the two-page HTML summary output, initialize the `Summarizer` class with the model object. Then pass in the filename, filepath, start date, and end date to `output_model_results_summary` to run the summary for that time duration and save it to the specified file.

In [ ]:
mmm_summarizer = summarizer.Summarizer(mmm)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
filepath = '/content/drive/MyDrive/Meridian_OPs/'
start_date = '2021-01-25'
end_date = '2024-01-15'
mmm_summarizer.output_model_results_summary(
    'Regional_prior_.5_std_.8.html', filepath, start_date, end_date
)

Here is a preview of the two-page output based on the simulated data:

In [ ]:
IPython.display.HTML(filename='/content/drive/MyDrive/Meridian_OPs/Regional_prior_.5_std_.8.html')

For a customized two-page report, model results summary table, and individual visualizations, see [Model results report](https://developers.google.com/meridian/docs/user-guide/generate-model-results-report) and [plot media visualizations](https://developers.google.com/meridian/docs/user-guide/plot-media-visualizations).





<a name="generate-optimize"></a>
## Step 5: Run budget optimization & generate an optimization report

You can choose what scenario to run for the budget allocation. In default scenario, you find the optimal allocation across channels for a given budget to maximize the return on investment (ROI).

1\. Instantiate the `BudgetOptimizer` class and run the `optimize()` method without any customization, to run the default library's Fixed Budget Scenario to maximize ROI.

In [ ]:
%%time
budget_optimizer = optimizer.BudgetOptimizer(mmm)
optimization_results = budget_optimizer.optimize()

2\. Export the 2-page HTML optimization report, which contains optimized spend allocations and ROI.

In [ ]:
filepath = '/content/drive/MyDrive/Meridian_OPs/'
optimization_results.output_optimization_summary(
    'Regional_prior_.5_std_.8.html', filepath
)

In [ ]:
IPython.display.HTML(filename='/content/drive/MyDrive/Meridian_OPs/Regional_prior_.5_std_.8.html')

For information about customized optimization scenarios, such as flexible budget scenarios, see [Budget optimization scenarios](https://developers.google.com/meridian/docs/user-guide/budget-optimization-scenarios). For more information about optimization results summary and individual visualizations, see [optimization results output](https://developers.google.com/meridian/docs/user-guide/generate-optimization-results-output) and [optimization visualizations](https://developers.google.com/meridian/docs/user-guide/plot-optimization-visualizations).

<a name="save-model"></a>
## Step 6: Save the model object

We recommend that you save the model object for future use. This helps you to  avoid repetitive model runs and saves time and computational resources. After the model object is saved, you can load it at a later stage to continue the analysis or visualizations without having to re-run the model.


Run the following codes to save the model object:

In [ ]:
file_path = '/content/drive/MyDrive/Meridian_models/Regional_prior_.5_std_.8.pkl'
model.save_mmm(mmm, file_path)

Run the following codes to load the saved model:

In [ ]:
mmm = model.load_mmm(file_path)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')